In [4]:
from itertools import product
from random import random, randint, shuffle, seed

import numpy as np
from scipy import sparse
from functools import reduce
from random import randint, seed
from copy import deepcopy
import pandas as pd

def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [5]:
def evaluate_fitness(state, problem_matrix):
    #Evaluate the fitness of a state
    global fitness_call_count
    fitness_call_count += 1

    cost = np.sum(state)
    coverage = np.sum(
        reduce(
            np.logical_or,
            [[problem_matrix[i,j] for j in range(TOTAL_POINTS)] for i, included in enumerate(state) if included],
            np.array([False for _ in range (TOTAL_POINTS)]),
        )
    )
    return coverage, -cost

def modify_state(current_state):
    #Generate a neighbor state by flipping one set's inclusion
    modified_state = deepcopy(current_state)
    random_index = randint(0, TOTAL_POINTS - 1)
    modified_state[random_index] = not modified_state[random_index]
    return modified_state



def run_hill_climbing(problem_matrix, iterations=1000):
    global fitness_call_count
    fitness_call_count = 0  # Reset fitness function call counter
    current_solution = [False] * TOTAL_SETS

    for _ in range(iterations):
        potential_solution = modify_state(current_solution)
        if evaluate_fitness(potential_solution, problem_matrix) >= evaluate_fitness(current_solution, problem_matrix):
            current_solution = potential_solution

    return current_solution, evaluate_fitness(current_solution, problem_matrix), fitness_call_count

def run_tabu_search(problem_matrix, iterations=1000, tabu_size=100):
    global fitness_call_count
    fitness_call_count = 0  # Reset fitness function call counter
    current_solution = [False] * TOTAL_SETS
    tabu_list = []

    for _ in range(iterations):
        potential_solution = modify_state(current_solution)
        if not potential_solution in tabu_list:
            if evaluate_fitness(potential_solution, problem_matrix) >= evaluate_fitness(current_solution, problem_matrix):
                # Update the tabu list with the current solution
                # if len(tabu_list) >= tabu_size:
                #     tabu_list.pop(0)  # Remove the oldest entry
                tabu_list.append(current_solution)
                current_solution = potential_solution
            else:
                tabu_list.append(potential_solution)

    return current_solution, evaluate_fitness(current_solution, problem_matrix), fitness_call_count


In [6]:
TOTAL_SETS = 100
TOTAL_POINTS = TOTAL_SETS
COVER_DENSITY = 0.3
fitness_call_count = 0  # Global variable to track fitness function calls


# Generate the set covering problem
problem_matrix = make_set_covering_problem(TOTAL_SETS, TOTAL_POINTS, COVER_DENSITY)

# Run hill climbing optimization
hill_climbing_solution, hill_climbing_fitness, hill_climbing_call_count = run_hill_climbing(problem_matrix)

print(f"Hill Climbing - Number of fitness function calls: {hill_climbing_call_count-1}")
print(f"Hill Climbing - Solution fitness: {hill_climbing_fitness}")

# Run tabu search optimization
tabu_search_solution, tabu_search_fitness, tabu_search_call_count = run_tabu_search(problem_matrix)

print(f"Tabu Search - Number of fitness function calls: {tabu_search_call_count-1}")
print(f"Tabu Search - Solution fitness: {tabu_search_fitness}")
print()

Hill Climbing - Number of fitness function calls: 2000
Hill Climbing - Solution fitness: (100, -8)
Tabu Search - Number of fitness function calls: 328
Tabu Search - Solution fitness: (100, -9)



In [7]:
TOTAL_SETS = 100
TOTAL_POINTS = TOTAL_SETS
COVER_DENSITY = 0.7
fitness_call_count = 0  # Global variable to track fitness function calls


# Generate the set covering problem
problem_matrix = make_set_covering_problem(TOTAL_SETS, TOTAL_POINTS, COVER_DENSITY)

# Run hill climbing optimization
hill_climbing_solution, hill_climbing_fitness, hill_climbing_call_count = run_hill_climbing(problem_matrix)

print(f"Hill Climbing - Number of fitness function calls: {hill_climbing_call_count-1}")
print(f"Hill Climbing - Solution fitness: {hill_climbing_fitness}")

# Run tabu search optimization
tabu_search_solution, tabu_search_fitness, tabu_search_call_count = run_tabu_search(problem_matrix)

print(f"Tabu Search - Number of fitness function calls: {tabu_search_call_count-1}")
print(f"Tabu Search - Solution fitness: {tabu_search_fitness}")
print()

Hill Climbing - Number of fitness function calls: 2000
Hill Climbing - Solution fitness: (100, -3)
Tabu Search - Number of fitness function calls: 386
Tabu Search - Solution fitness: (100, -3)

